# Importing Libraries

In [1]:
import torch 
from torch import nn
from torch.utils.data import DataLoader
from torch import optim
from torchvision import transforms, datasets
import torchvision

from torch.utils.tensorboard import SummaryWriter
import comet_ml
#from comet_ml import Experiment

In [2]:
# Better stablity 

# Hyperparamters 
device = "cuda" if torch.cuda.is_available() else "cpu"
lr = 5e-5
imageSize = 28
imageChannel = 1 
latentDim = 100
epochs = 5
featuresCritic = 64
featuresGen = 64
criticIteration = 5
weightClip = 0.01  
batchSize = 32

In [3]:
# Transforms 
transforms = transforms.Compose([
    transforms.Resize(imageSize),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.5 for i in range(imageChannel)],[0.5 for i in range(imageChannel)]
    )
])

# Dataset and Dataloader Class

In [4]:
dataset = datasets.MNIST("dataset/", transform=transforms, download=False)
dataloader = DataLoader(dataset, batch_size=batchSize, shuffle=True)

In [5]:
# for idx, (data, label) in enumerate(dataloader):
#     print(f"Data Shape: {data.shape} and Label: {label} with shape {label.shape}")
#     if idx==2:
#         break

# Class Generator

In [6]:
class Generator(nn.Module):
    def __init__(self,channelNoise, channelImg, features) -> None:
        super(Generator, self).__init__();
        # featuresGen is 64 so here features will be 1024/64 according to DCGAN.
        self.GenBlock = nn.Sequential(
            self._genBlock(channelNoise, features*16, 4, 1, 0),
            self._genBlock(features*16, features*8, 4, 2, 1),
            self._genBlock(features*8, features*4, 4, 2, 1),
            self._genBlock(features*4, features*2, 4, 2, 1),
            self._lastBlock(features*2, channelImg, 4, 2, 1)
        )

    def _genBlock(self,inFeatures, outFeatures, kernel, stride, padding):
        self.block = nn.Sequential(
            nn.ConvTranspose2d(inFeatures, outFeatures, kernel_size=kernel, stride=stride, padding=padding),
            nn.BatchNorm2d(outFeatures),
            nn.LeakyReLU(0.2)
        )
        return self.block

    def _lastBlock(self,inFeatures, outFeatures, kernel, stride, padding):
        self.block = nn.Sequential(
            nn.ConvTranspose2d(inFeatures, outFeatures, kernel, stride, padding),
            nn.Tanh()
        )
        return self.block
    
    def forward(self,x):
        return self.GenBlock(x)


In [7]:
# j = Generator(100, 1 , 64)
# j1 = j(torch.randn(32,100,1,1))
# j1.shape

# Class Discriminator

In [8]:
class Discriminator(nn.Module):
    def __init__(self, channelImg, features) -> None:
        super(Discriminator, self).__init__();
        self.blockDisc = nn.Sequential(
            nn.Conv2d(channelImg, features, 4, 2, 1),
            nn.BatchNorm2d(features),
            nn.LeakyReLU(0.2),

            nn.Conv2d(features, features*2, 4, 2, 1),
            nn.BatchNorm2d(features*2),
            nn.LeakyReLU(0.2),

            nn.Conv2d(features*2, features*4, 4, 2, 1),
            nn.BatchNorm2d(features*4),
            nn.LeakyReLU(0.2),

            nn.Conv2d(features*4, features*8, 4, 2, 1),
            nn.BatchNorm2d(features*8),
            nn.LeakyReLU(0.2),

            nn.Conv2d(features*8, 1 , 4, 1, 0)
        )

    def forward(self, x):
        return self.blockDisc(x)

In [9]:
# j = Discriminator(1,64)
# j1 = j(torch.randn(32,1,64,64)).reshape(-1)
# j1.shape

# Objects for Class Generator and Class Discriminator

In [10]:
gen = Generator(channelNoise=latentDim, channelImg=imageChannel, features=featuresGen).to(device)
disc = Discriminator(channelImg=imageChannel, features=featuresCritic).to(device)
print("Okay with objects formation!!")

Okay with objects formation!!


# Optimizers 

In [11]:
optmGen = optim.RMSprop(gen.parameters(), lr=lr)
optmDisc = optim.RMSprop(disc.parameters(), lr=lr)

In [12]:
# Fixed Noise\
fixedNoise = torch.randn(batchSize, latentDim, 64, 64).to(device)

# Training Loop

In [13]:
print("Starting Training.....!!!")
for i in range(0,epochs):
    for batchIndx, (real, _) in enumerate(dataloader):
        real = real.to(device)

        for _ in range(criticIteration):
            noise = torch.randn(batchSize, latentDim, 1, 1).to(device)
            print("Noise Shape: ",noise.shape)
            fake = gen(noise)
            criticReal = disc(real).reshape(-1)
            criticFake = disc(fake).reshape(-1)

            lossCritic = -(torch.mean(criticReal) - torch.mean(criticFake))
            disc.zero_grad()
            lossCritic.backward(retain_graph=False)
            optmDisc.step()

            for p in disc.parameters():
                p.data.clamp(-weightClip, weightClip)

        
        ## Train Generator: min -E[critic(fake)]
        fake1 = gen(noise)
        output = disc(fake1).reshape(-1)
        lossGen = -torch.mean(output)
        gen.zero_grad()
        lossGen.backward()
        optmGen.step()

        with torch.no_grad():
            fakeInputToGen = gen(fixedNoise)
            imgGridReal = torchvision.utils.make_grid(real[:batchSize], normalize=True)
            imgGridFake = torchvision.utils.make_grid(fakeInputToGen[:batchSize], normalize=True)

Starting Training.....!!!
Noise Shape:  torch.Size([32, 100, 1, 1])


RuntimeError: Calculated padded input size per channel: (1 x 1). Kernel size: (4 x 4). Kernel size can't be greater than actual input size